# 线性模型

## 模型
$$\hat{y} = w_1x_1 + w_2x_2 + \dots + w_nx_n + b\tag{1} $$

## 损失函数
$$ L = \frac{1}{2}\sum_{i=1}^n(y_i - \hat{y_i})^2\tag{2} $$

准备数据,数据有100条记录，每条记录有两个特征$x_1$和$x_2$

In [573]:
import numpy as np
import seaborn as sns
import pandas as pd

In [574]:
true_w = np.array([[1.4], [2.3]], dtype = np.float64)
true_b = 4.7

In [575]:
X = np.random.normal(0, 1, (100,2))
zao = np.random.normal(0, 0.001,(100,1))
y = np.dot(X, true_w) + true_b + zao

现在我们有了数据，特征$X$和对应的关注值$y$，我们要如何去得到线性模型的参数$w$和$b$呢?  
若记$\bar{w}=\left[\begin{matrix}w \\ b \end{matrix}\right]$,$\bar{X}=\left[\begin{matrix}X & 1 \end{matrix}\right]$对于统计学，你可以推导出来最优解 $\hat{\bar{w}}^*=(\bar{X}^\intercal\bar{X})^{-1}(\bar{X}^\intercal)y$。但是这个方法有很大的局限性  
1. X必须是可逆的
2. 当数据量和特征维度都很大是，计算耗时，求逆更耗时  

那么有别的办法吗？有！从优化的角度来做。这是个凸函数，用梯度下降的方法，一定可以找到全局最优解

首先计算损失函数对参数的偏导数  
$$\frac{\partial{L}}{\partial{w_1}}= \sum_{i=1}^n(\hat{y_i}-y_i)x_{i1}\tag{3}$$
$$\frac{\partial{L}}{\partial{b}}= \sum_{i=1}^n(\hat{y_i}-y_i)\tag{4}$$

在本例中，若定义$\vec{w}=\left[\begin{matrix}w1 \\ w2 \\ b \end{matrix}\right]$,那么  
$$\frac{\partial{L}}{\partial{\vec{w}}}= \sum_{i=1}^n(\hat{y_i}-y_i)\left[\begin{matrix}x_{i1}\\x_{i2}\\1 \end{matrix}\right]\tag{4}$$

写成矩阵形式，对$w$的计算为, $X_{100\times2}$ ,$y_{100\times1}$,$\hat{y}_{100\times1}$
$$\frac{\partial{L}}{\partial{\left[\begin{matrix}w_1\\w_2 \end{matrix}\right]}} = X^\intercal(\hat{y}-y)\tag{5}$$  
对$y$的计算为,其中$\vec{1}$元素全为1的列向量
$$\frac{\partial{L}}{\partial{y}} = {\vec{1}}^\intercal(\hat{y}-y)\tag{5}$$ 

先随机初始化$w$和$b$

In [576]:
w = np.random.randn(2,1)
b = np.random.randn(1)
w,b

(array([[0.69316828],
        [1.49954833]]), array([0.78372433]))

根据推导的公式计算梯度

In [577]:
def get_grad(X, y, w, b):
    return np.dot(X.T,np.dot(X,w)+b-y), np.dot(np.ones((1,X.shape[0])),np.dot(X,w)+b-y)

In [578]:
get_grad(X, y, w, b)

(array([[-85.79259115],
        [-48.88827012]]), array([[-387.33870906]]))

而深度学习框架都提供了自动求导机制，省去了手动推导公式的繁琐，对于神经网络等更有用

## Mxnet

In [579]:
from mxnet import ndarray as nd, autograd
ndX = nd.array(X)
ndy = nd.array(y)
ndzao = nd.array(zao)
ndtrue_w = nd.array(true_w)
ndtrue_b = nd.array([true_b])
ndw = nd.array(w)
ndb = nd.array(b)

In [580]:
#mxnet的变量自动求导，需要先给其分配内存记录
ndw.attach_grad()
ndb.attach_grad()

In [581]:
with autograd.record():
    los = (ndy-(nd.dot(ndX, ndw) +  ndb))**2/2

In [582]:
los.backward()

In [583]:
ndw.grad,ndb.grad

(
 [[-85.792595]
  [-48.888275]]
 <NDArray 2x1 @cpu(0)>, 
 [-387.3387]
 <NDArray 1 @cpu(0)>)

可以发现，使用mxnet的自动求导机制算得的梯度，和我们手动计算的梯度是一致的

## Pytorch

## 利用梯度下降训练模型，这里先定义几个函数 

损失函数

In [584]:
def loss(X, y, w, b):
    return 1/2*(np.dot(X,w)+b-y)**2

求梯度函数

In [585]:
def get_grad(X, y, w, b):
    return np.dot(X.T,np.dot(X,w)+b-y), np.dot(np.ones((1,X.shape[0])),np.dot(X,w)+b-y)

In [586]:
lr = 0.3
num_epochs = 100
N = X.shape[0]
for i in range(num_epochs):
    w_grad, b_grad = get_grad(X,y,w,b)
    w = w - lr*w_grad/N
    b = b - lr*b_grad/N
    los = loss(X, y, w, b)
    if not i % 10:
        print(i,'th iter,','loss is:', np.mean(los))

0 th iter, loss is: 4.005747715722106
10 th iter, loss is: 0.004760808917494303
20 th iter, loss is: 1.1206034496498005e-05
30 th iter, loss is: 4.4904229915226564e-07
40 th iter, loss is: 4.0541935284359094e-07
50 th iter, loss is: 4.051967880069255e-07
60 th iter, loss is: 4.0519558993317587e-07
70 th iter, loss is: 4.051955834153209e-07
80 th iter, loss is: 4.051955833797791e-07
90 th iter, loss is: 4.051955833795651e-07


In [587]:
w,b

(array([[1.39998999],
        [2.29991362]]), array([[4.70003541]]))

In [588]:
true_w,true_b

(array([[1.4],
        [2.3]]), 4.7)

## 下面实现随机梯度下降法来计算模型参数  
首先分割数据集

In [589]:
def data_iter1(batch_size, X, y):
    num_examples = len(X)
    indices = list(range(num_examples))
    np.random.shuffle(indices)  # 样本的读取顺序是随机的。
    for i in range(0, num_examples, batch_size):
        j = np.array(indices[i: min(i + batch_size, num_examples)])
        yield X[j], y[j]  # take 函数根据索引返回对应元素。

In [590]:
batch_size = 10
for i,j in data_iter(batch_size, X, y):
    print(i,j)
    break

[[-0.3233138  -1.4846215 ]
 [ 0.44555086 -1.35650448]
 [ 0.82450613  0.14168134]
 [ 0.32114495 -0.18607105]
 [ 0.78150391 -0.26899311]
 [-2.14545196  0.44561945]
 [ 0.32489975 -0.43525001]
 [-0.13669077 -0.03272827]
 [ 0.61720557 -0.42424093]
 [ 1.22565425 -0.49575901]] [[0.8316827 ]
 [2.203336  ]
 [6.17885369]
 [4.72081294]
 [5.1757113 ]
 [2.72212542]
 [4.15458222]
 [4.4321947 ]
 [4.58706081]
 [5.27548265]]


重新初始化$w$,$b$

In [591]:
w = np.random.randn(2,1)
b = np.random.randn(1,1)

In [592]:
lr = 0.3
num_epochs = 100
N = X.shape[0]
for i in range(num_epochs):
    for X_s,y_s in data_iter(batch_size, X, y):
        w_grad, b_grad = get_grad(X_s,y_s,w,b)
        w = w - lr*w_grad/batch_size
        b = b - lr*b_grad/batch_size
    los = loss(X, y, w, b)
    if not i % 10:
        print(i,'th iter,','loss is:', np.mean(los))

0 th iter, loss is: 0.012799319293725298
10 th iter, loss is: 4.431399113513943e-07
20 th iter, loss is: 4.06262070265052e-07
30 th iter, loss is: 4.1230349422638e-07
40 th iter, loss is: 4.123345896880925e-07
50 th iter, loss is: 4.304242828911277e-07
60 th iter, loss is: 4.141127151582546e-07
70 th iter, loss is: 4.0703805816307525e-07
80 th iter, loss is: 4.060119543184918e-07
90 th iter, loss is: 4.0733200330309745e-07


In [593]:
w,b

(array([[1.39988949],
        [2.29995442]]), array([[4.70001375]]))

## 使用Mxnet来训练模型

In [594]:
def net(X, w, b):
    return nd.dot(X,w)+b
def myloss(y,yhat):
    return (y-yhat)**2/2
def sgd(params, lr, batch_size):  # 本函数已保存在 gluonbook 包中方便以后使用。
    for param in params:
        param[:] = param - lr * param.grad / batch_size

In [595]:
ndw = nd.random.randn(2,1)
ndb = nd.random.randn(1,1)

In [596]:
ndw, ndb

(
 [[-1.5918757]
  [-1.1081947]]
 <NDArray 2x1 @cpu(0)>, 
 [[0.0787202]]
 <NDArray 1x1 @cpu(0)>)

In [597]:
lr = 0.3
num_epochs = 100
N = ndX.shape[0]
ndw.attach_grad()
ndb.attach_grad()

这里有一个注意点，如果更新参数$ndw$时使用$ndw$而不是使用$ndw[:]$，需要重新申请存储梯度的内存，这是因为ndw已经指向了新的内存空间，这个变量之前申请的存储梯度内存已经失效。而$ndw[:]$的值则是写入了原来变量的内存空间，所以不需要再次申请存储梯度的内存空间（多说一句，这和python的list有所不同，$a = [1,2,3,4]$，$a[:]$是指向了新的内存空间）

In [598]:
for i in range(num_epochs):
    with autograd.record():
        los = (ndy-(nd.dot(ndX, ndw) +  ndb))**2/2
    los.backward()
    ndw[:] = ndw-lr*ndw.grad/N
    ndb[:] = ndb-lr*ndb.grad/N
    '''ndw = ndw-lr*ndw.grad/N
    ndb = ndb-lr*ndb.grad/N
    ndw.attach_grad()
    ndb.attach_grad()'''
    
    if not i % 10:
        print(i,'th iter,','loss is:', ((ndy-(nd.dot(ndX, ndw) +  ndb))**2/2).mean().asnumpy())

0 th iter, loss is: [10.221592]
10 th iter, loss is: [0.01048965]
20 th iter, loss is: [1.1256972e-05]
30 th iter, loss is: [4.167761e-07]
40 th iter, loss is: [4.0519186e-07]
50 th iter, loss is: [4.0520598e-07]
60 th iter, loss is: [4.0520598e-07]
70 th iter, loss is: [4.0520598e-07]
80 th iter, loss is: [4.0520598e-07]
90 th iter, loss is: [4.0520598e-07]


In [599]:
ndw, ndb

(
 [[1.3999902]
  [2.2999134]]
 <NDArray 2x1 @cpu(0)>, 
 [[4.7000346]]
 <NDArray 1x1 @cpu(0)>)

## Mxnet的随机梯度下降实现

In [600]:
def data_iter2(batch_size, X, y):
    num_examples = len(X)
    indices = list(range(num_examples))
    np.random.shuffle(indices)  # 样本的读取顺序是随机的。
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i: min(i + batch_size, num_examples)])
        yield X.take(j), y.take(j)  # take 函数根据索引返回对应元素。

In [601]:
batch_size = 10
for i,j in data_iter2(batch_size, ndX, ndy):
    print(i,j)
    break


[[ 0.4681375  -0.15534014]
 [ 0.26530302  0.3404787 ]
 [ 0.42121166 -1.0664554 ]
 [-1.551066   -0.38270727]
 [ 1.3770654   0.13794978]
 [ 0.5307424   0.35908428]
 [ 0.8445848  -0.54597986]
 [ 0.11354028 -0.4180504 ]
 [ 0.07860706 -1.2999861 ]
 [ 1.1178277  -0.14915887]]
<NDArray 10x2 @cpu(0)> 
[[4.9992514]
 [5.8550897]
 [2.8376958]
 [1.6484268]
 [6.9457483]
 [6.2704363]
 [4.6253896]
 [3.8988945]
 [1.8206441]
 [5.9241266]]
<NDArray 10x1 @cpu(0)>


重新初始化ndw和ndb

In [602]:
ndw = nd.random.randn(2,1)
ndb = nd.random.randn(1,1)
ndw.attach_grad()
ndb.attach_grad()

In [603]:
lr = 0.03
num_epochs = 20
net = linreg
loss = squared_loss

for epoch in range(num_epochs):  # 训练模型一共需要 num_epochs 个迭代周期。
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。
    # X 和 y 分别是小批量样本的特征和标签。
    for X, y in data_iter(batch_size, ndX, ndy):
        with autograd.record():
            l = loss(net(X, ndw, ndb), y)  # l 是有关小批量 X 和 y 的损失。
        l.backward()  # 小批量的损失对模型参数求梯度。
        sgd([ndw, ndb], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数。
    train_l = loss(net(ndX, ndw, ndb), ndy)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 13.256109
epoch 2, loss 7.338350
epoch 3, loss 4.063630
epoch 4, loss 2.248680
epoch 5, loss 1.245518
epoch 6, loss 0.690749
epoch 7, loss 0.383504
epoch 8, loss 0.212836
epoch 9, loss 0.118251
epoch 10, loss 0.065795
epoch 11, loss 0.036639
epoch 12, loss 0.020418
epoch 13, loss 0.011393
epoch 14, loss 0.006371
epoch 15, loss 0.003570
epoch 16, loss 0.002003
epoch 17, loss 0.001125
epoch 18, loss 0.000634
epoch 19, loss 0.000358
epoch 20, loss 0.000202


In [604]:
ndw,ndb

(
 [[1.4003743]
  [2.2887452]]
 <NDArray 2x1 @cpu(0)>, 
 [[4.682356]]
 <NDArray 1x1 @cpu(0)>)

换一种参数初始化方法

In [605]:
ndw = nd.random.normal(0,0.001,(2,1))
ndb = nd.zeros((1,1))
ndw.attach_grad()
ndb.attach_grad()

In [606]:
lr = 0.03
num_epochs = 20
net = linreg
loss = squared_loss

for epoch in range(num_epochs):  # 训练模型一共需要 num_epochs 个迭代周期。
    # 在每一个迭代周期中，会使用训练数据集中所有样本一次（假设样本数能够被批量大小整除）。
    # X 和 y 分别是小批量样本的特征和标签。
    for X, y in data_iter(batch_size, ndX, ndy):
        with autograd.record():
            l = loss(net(X, ndw, ndb), y)  # l 是有关小批量 X 和 y 的损失。
        l.backward()  # 小批量的损失对模型参数求梯度。
        sgd([ndw, ndb], lr, batch_size)  # 使用小批量随机梯度下降迭代模型参数。
    train_l = loss(net(ndX, ndw, ndb), ndy)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 7.829946
epoch 2, loss 4.346745
epoch 3, loss 2.414821
epoch 4, loss 1.343420
epoch 5, loss 0.746863
epoch 6, loss 0.415598
epoch 7, loss 0.232045
epoch 8, loss 0.129735
epoch 9, loss 0.072646
epoch 10, loss 0.040719
epoch 11, loss 0.022895
epoch 12, loss 0.012895
epoch 13, loss 0.007277
epoch 14, loss 0.004117
epoch 15, loss 0.002338
epoch 16, loss 0.001330
epoch 17, loss 0.000760
epoch 18, loss 0.000435
epoch 19, loss 0.000250
epoch 20, loss 0.000144


In [607]:
ndw, ndb

(
 [[1.4035972]
  [2.2903054]]
 <NDArray 2x1 @cpu(0)>, 
 [[4.685209]]
 <NDArray 1x1 @cpu(0)>)

## 使用Mxnet的高阶API Gluon来实现

回忆一下模型的三要素
1. 模型
2. 损失函数
3. 优化算法  
  
这几者在Gluon中都有对应

获取数据，Gluon有data模块来将数据分批

In [609]:
from mxnet.gluon import data as gdata

In [610]:
??gdata

In [614]:
?? gdata.ArrayDataset

In [615]:
?? gdata.DataLoader

In [619]:
batch_size = 10
dataset = gdata.ArrayDataset(ndX, ndy)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

### 定义模型

In [637]:
from mxnet.gluon import nn

In [638]:
net = nn.Sequential()
net.add(nn.Dense(1))

### 初始化模型参数

In [639]:
from mxnet import init
?? init.Normal

In [640]:
net.initialize(init.Normal(sigma=0.01))

### 定义损失函数

In [641]:
from mxnet.gluon import loss as gloss

In [642]:
?? gloss

In [643]:
loss = gloss.L2Loss()

### 定义优化算法

In [644]:
from mxnet import gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

###  训练模型

In [645]:
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
        l = loss(net(ndX), ndy)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 7.844169
epoch 2, loss: 4.352446
epoch 3, loss: 2.417657
epoch 4, loss: 1.342142
epoch 5, loss: 0.747396
epoch 6, loss: 0.416652
epoch 7, loss: 0.232186
epoch 8, loss: 0.129725
epoch 9, loss: 0.072532
epoch 10, loss: 0.040671
epoch 11, loss: 0.022841
epoch 12, loss: 0.012856
epoch 13, loss: 0.007256
epoch 14, loss: 0.004097
epoch 15, loss: 0.002324
epoch 16, loss: 0.001322
epoch 17, loss: 0.000754
epoch 18, loss: 0.000431
epoch 19, loss: 0.000248
epoch 20, loss: 0.000143


In [646]:
ndtrue_w, ndtrue_b

(
 [[1.4]
  [2.3]]
 <NDArray 2x1 @cpu(0)>, 
 [4.7]
 <NDArray 1 @cpu(0)>)

In [649]:
dense = net[0]

In [652]:
dense.weight.data()


[[1.403397  2.2903547]]
<NDArray 1x2 @cpu(0)>

In [653]:
dense.bias.data()


[4.6852536]
<NDArray 1 @cpu(0)>